In [1]:
import os
import re
import pandas as pd
from tqdm import tqdm
from transformers import pipeline
import torch
import unicodedata

In [8]:

# === A. Ayarlar ===
klasor_yolu = "./yorum_dosyaları"
etiketler = ["eleştirel", "şaka amaçlı", "bilgilendirici", "fanboy/fangirl", "nötr"]
batch_size = 8  # yorumları kaçlı gruplar halinde işler

In [9]:
# === B. GPU kontrolü
cihaz = 0 if torch.cuda.is_available() else -1
print(f"⚙️ Kullanılan cihaz: {'GPU' if cihaz == 0 else 'CPU'}")

⚙️ Kullanılan cihaz: GPU


In [10]:
# === C. Zero-shot pipeline
classifier = pipeline(
    "zero-shot-classification",
    model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli",
    device=cihaz
)

Device set to use cuda:0


In [11]:
# === D. Şaka olup olmadığını belirleyen fonksiyon (Unicode destekli)
def is_potential_joke(text, model_label=None, model_score=1.0):
    special_chars = [
        ch for ch in str(text)
        if not ch.isalnum() and not ch.isspace() and unicodedata.category(ch)[0] != "L"
    ]
    return len(special_chars) > 10 and model_score < 0.4

In [13]:
# === E. Dosyaları işle
for dosya in os.listdir(klasor_yolu):
    if dosya.endswith(".csv"):
        print(f"\n📂 Dosya işleniyor: {dosya}")
        try:
            df = pd.read_csv(os.path.join(klasor_yolu, dosya), comment="#")
            df = df[["review_text"]].dropna()
            df = df[df["review_text"].str.len() > 10].reset_index(drop=True)

            playtime_var = "playtime_forever_minutes" in df.columns
            if playtime_var:
                df = df[df["playtime_forever_minutes"].notna()].reset_index(drop=True)

            print(f"🗂 Etiketlenecek yorum sayısı: {len(df)}")

            predicted_labels = []
            confidences = []
            playtimes = df["playtime_forever_minutes"].tolist() if playtime_var else [None] * len(df)
            texts = df["review_text"].tolist()

            # === Toplu işlem
            for i in tqdm(range(0, len(texts), batch_size), desc="Etiketleniyor..."):
                batch_texts = texts[i:i+batch_size]
                batch_ptimes = playtimes[i:i+batch_size]

                results = classifier(
                    batch_texts,
                    candidate_labels=etiketler,
                    multi_label=False
                )

                for j in range(len(batch_texts)):
                    label = results[j]["labels"][0]
                    score = round(results[j]["scores"][0], 2)

                    # Özel karakter + skor değerlendirmesi
                    if is_potential_joke(batch_texts[j], label, score):
                        label = "şaka amaçlı"
                        score = 0.95

                    # Oynama süresine göre etiket düzeltmesi
                    play = batch_ptimes[j]
                    if playtime_var and play is not None:
                        if play < 15 and label not in ["şaka amaçlı", "nötr"]:
                            label = "nötr"
                            score = round(score * 0.8, 2)
                        elif play > 500 and label == "nötr":
                            label = "fanboy/fangirl"
                            score = round(score * 0.8, 2)

                    predicted_labels.append(label)
                    confidences.append(score)

            # === Sonuçları ekle ve kaydet
            df["predicted_label"] = predicted_labels
            df["confidence"] = confidences

            cikti_adi = f"etiketli_{os.path.splitext(dosya)[0]}.csv"
            df.to_csv(cikti_adi, index=False)
            print(f"✅ Kaydedildi: {cikti_adi}")

        except Exception as e:
            print(f"❌ Hata oluştu ({dosya}): {e}")


📂 Dosya işleniyor: Age_of_Empires_IV_Anniversary_Edition_yorumlar.csv
🗂 Etiketlenecek yorum sayısı: 6671


Etiketleniyor...: 100%|██████████| 834/834 [17:34<00:00,  1.26s/it]


✅ Kaydedildi: etiketli_Age_of_Empires_IV_Anniversary_Edition_yorumlar.csv

📂 Dosya işleniyor: ARK_yorumlar.csv
🗂 Etiketlenecek yorum sayısı: 6081


Etiketleniyor...: 100%|██████████| 761/761 [18:12<00:00,  1.44s/it]


✅ Kaydedildi: etiketli_ARK_yorumlar.csv

📂 Dosya işleniyor: Balatro_yorumlar.csv
🗂 Etiketlenecek yorum sayısı: 7737


...

✅ Kaydedildi: etiketli_The_Sims_4_yorumlar.csv

📂 Dosya işleniyor: The_Witcher_3_yorumlar.csv
🗂 Etiketlenecek yorum sayısı: 6899


Etiketleniyor...: 100%|██████████| 863/863 [25:31<00:00,  1.77s/it]


✅ Kaydedildi: etiketli_The_Witcher_3_yorumlar.csv

📂 Dosya işleniyor: Total_War_WARHAMMER_II_yorumlar.csv
🗂 Etiketlenecek yorum sayısı: 7120


Etiketleniyor...: 100%|██████████| 890/890 [24:39<00:00,  1.66s/it]

✅ Kaydedildi: etiketli_Total_War_WARHAMMER_II_yorumlar.csv
